# By-Band _g_-Point Reduction

In [1]:
%load_ext autoreload
%autoreload 2

# Dependencies

`numpy` is installed in the Python environment at NERSC (`module load python`), but `xarray` is not, so the user must install the package on their own. `PIPPATH` is the assumed location. This notebook depends heavily on `xarray`.

In [2]:
# standard libraries
import os, sys, shutil, glob, pickle
from multiprocessing import Pool

# "standard" conda install (NERSC already provides this without user 
# having to install it)
import numpy as np

# conda installs
from tqdm import tqdm

# directory in which libraries installed with conda are saved
PIPPATH = '{}/.local/'.format(os.path.expanduser('~')) + \
    'cori/3.7-anaconda-2019.10/lib/python3.7/site-packages'
PIPPATH = '/global/homes/e/emlawer/.local/cori/3.8-anaconda-2020.11/' + \
     'lib/python3.8/site-packages'
PIPPATH = '/global/homes/p/pernak18/.local/cori/3.9-anaconda-2021.11/lib/python3.9/site-packages'
PIPPATH = '/global/common/software/nersc/cori-2022q1/sw/python/3.9-anaconda-2021.11'
PATHS = ['common']
for path in PATHS: sys.path.append(path)

# needed at AER unless i update `pandas`
import warnings
#warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

# user must do `pip install xarray` on cori (or other NERSC machines)
import xarray as xa

# local modules
import g_point_reduction as REDUX
import modified_reduction as MODRED
from rrtmgp_cost_compute import flux_cost_compute as FCC

# Static Inputs

In [3]:
# only do one domain or the other
DOLW = True
DOMAIN = 'LW' if DOLW else 'SW'
NBANDS = 16 if DOLW else 14

# does band-splitting need to be done, or are there existing files 
# that have divided up the full k-distribution?
BANDSPLIT = False

# Paths

In [4]:
PROJECT = '/global/project/projectdirs/e3sm/pernak18/'
EXE = '{}/g-point-reduction/garand_atmos/rrtmgp_garand_atmos'.format(
    PROJECT)
REFDIR = '{}/reference_netCDF/g-point-reduce'.format(PROJECT)

# test (RRTMGP) and reference (LBL) flux netCDF files, full k-distributions, 
# and by-band Garand input file
fluxSuffix = 'flux-inputs-outputs-garandANDpreind.nc'
if DOLW:
    GARAND = '{}/multi_garand_template_single_band.nc'.format(REFDIR)
    KFULLNC = '{}/rrtmgp-data-lw-g256-2018-12-04.nc'.format(REFDIR)
    KFULLNC = '{}/rrtmgp-data-lw-g256-jen-xs.nc'.format(REFDIR)
    REFNC = '{}/lblrtm-lw-{}'.format(REFDIR, fluxSuffix)
    TESTNC = '{}/rrtmgp-lw-{}'.format(REFDIR, fluxSuffix)
    #TESTNC = 'rrtmgp-lw-flux-inputs-outputs-garand-all.nc'
else:
    GARAND = '{}/charts_multi_garand_template_single_band.nc'.format(REFDIR)
    KFULLNC = '{}/rrtmgp-data-sw-g224-2018-12-04.nc'.format(REFDIR)
    REFNC = '{}/charts-sw-{}'.format(REFDIR, fluxSuffix)
    TESTNC = '{}/rrtmgp-sw-{}'.format(REFDIR, fluxSuffix)
# endif LW

BANDSPLITDIR = 'band_k_dist'
FULLBANDFLUXDIR = 'full_band_flux'

for PATH in PATHS: FCC.pathCheck(PATH)

CWD = os.getcwd()

KPICKLE = '{}_k-dist.pickle'.format(DOMAIN)
pickleCost = '{}_cost-optimize.pickle'.format(DOMAIN)

# Band Splitting

Break up full _k_-distribution file into separate distributions for each band, then calculate the corresponding fluxes. This should only need to be run once.

After some clarifications from Robert (30-Nov-2020), I believe the plan of action is:

1. create Nbands k-distribution files
2. drive the Fortran executable Nbands times to produce Nbands flux results
3. the trial g-point combinations then loop over bands and the possible g-point combinations within each band, creating k-distribution and band-wise flux files for each possible combination
4. The Python code assembles broadband fluxes from the band-wise flux files in order to compute the cost functions

In [5]:
if BANDSPLIT:
    print('Band splitting commenced')
    FCC.pathCheck(BANDSPLITDIR, mkdir=True)
    FCC.pathCheck(FULLBANDFLUXDIR, mkdir=True)
    kFiles, fullBandFluxes = [], []
    for iBand in tqdm(range(NBANDS)):
        # divide full k-distribution into subsets for each band
        kObj = REDUX.gCombine_kDist(KFULLNC, iBand, DOLW, 1, 
            fullBandKDir=BANDSPLITDIR, fullBandFluxDir=FULLBANDFLUXDIR, 
            profilesNC=GARAND)
        kFiles.append(kObj.kBandNC)
        kObj.kDistBand()

        # quick, non-parallelized flux calculations (because the 
        # executable is run in one directory)
        FCC.fluxCompute(kObj.kBandNC, kObj.profiles, kObj.exe, 
                           kObj.fullBandFluxDir, kObj.fluxBandNC)
        fullBandFluxes.append(kObj.fluxBandNC)
    # end band loop
    print('Band splitting completed')
else:
    kFiles = sorted(glob.glob('{}/coefficients_{}_band??.nc'.format(
        BANDSPLITDIR, DOMAIN)))
    fullBandFluxes = sorted(glob.glob('{}/flux_{}_band??.nc'.format(
        FULLBANDFLUXDIR, DOMAIN)))

    if len(kFiles) == 0 or len(fullBandFluxes) == 0:
        print('WARNING: set `BANDSPLIT` to `True` and run this cell again')
# endif BANDSPLIT


# Pressure Levels for Cost Function

Pressure levels [Pa] for the Garand atmospheres are printed to standard output with indices that can be used in the cost function:

In [ ]:
with xa.open_dataset(REFNC) as refDS:
    pLev = refDS['p_lev'].isel(record=0)
for iLev, pLev in enumerate(pLev.isel(col=0).values): print(iLev, pLev)

# _g_-Point Combining

Combine _g_-point reduced for bands with full-band fluxes from other bands, find optimal _g_-point combination for given iteration, proceed to next iteration.

First, find all _g_-point combinations for each band. Store the band object in a dictionary for use in flux computation. This cell only needs to be run once, and to save time in development, the dictionary is saved in a `pickle` file and can be loaded in the next cell.

In [ ]:
# this should be parallelized; also is part of preprocessing so we 
# shouldn't have to run it multiple times
kBandDict = {}
for iBand, kFile in tqdm(enumerate(kFiles)):
    #if iBand != 0: continue
    band = iBand + 1
    kObj = REDUX.gCombine_kDist(kFile, iBand, DOLW, 1, 
        fullBandKDir=BANDSPLITDIR, 
        fullBandFluxDir=FULLBANDFLUXDIR)
    kObj.gPointCombine()
    kBandDict['band{:02d}'.format(band)] = kObj

    print('Band {} complete'.format(band))
# end kFile loop

import pickle
with open(KPICKLE, 'wb') as fp: pickle.dump(kBandDict, fp)

Now compute fluxes in parallel for every _g_-point combination -- merging occurs in each band, and these combinations in a given band are used with broadband fluxes from other bands. These concatenations each have an associated `xarray` dataset assigned to it. Cost function components are then calculated based for each dataset, and the one that minimizes the error in the cost function will have its associated netCDF saved to disk.

Uncomment pickling block to restore dictionary from previous cell.

# Reduction and Optimization

Test and reference netCDF files have flux and heating rate arrays of dimension `record` x `col` x `lay`/`lev` and `band` if the array is broken down by band. `record` represents atmospheric specifications that can be used in [forcing scenarios](https://github.com/pernak18/g-point-reduction/wiki/LW-Forcing-Number-Convention#g-point-reduction-convention-).

Alternatively, the atmospheric specifications from any scenario can also be used. "Bare" parameters like `heating_rate` and `flux_net` will be treated as PD specifications, so the user will have to specify explicitly if they want the fluxes or heating rates from other scenarios by using the `flux_*_N` and `heating_rate_N` convention, where `N` is the scenario index as listed in the above list. The same convention applies to band fluxes and HRs. `N` = 0 will work just like `heating_rate` and `flux_net`.

Forcing for this exercise is defined as PI subtracted from scenario (2-6). The convention for these quantities is `*_forcing_N`, where `*` is the typical flux or heating rate (band or broadband) string, and `N` again is the forcing scenario (`N` of 2 would be forcing due to doubling methane).

First, let's define the cost function (component names, levels/layers, and weights):

In [ ]:
# pickling for developement purposes so this dictionary doesn't need 
# to be regenerated for every code change.
with open(KPICKLE, 'rb') as fp: kBandDict = pickle.load(fp)

# components used in cost function computation
# variable names in RRTMGP and LBL flux netCDF file, except for 
# forcing, which has to be specifed with "_forcing" appended to 
# the appropriate array. e.g., "flux_net_forcing" for net flux forcing
# netCDF arrays ('heating_rate', 'flux_net', 'band_flux_net', etc.)
# or forcing scenarios: convention is  ('flux_net_forcing_3') for 
#CFCOMPS = ['flux_dif_net', 'flux_dir_dn', 'heating_rate']
CFCOMPS = ['flux_net','band_flux_net','heating_rate','heating_rate_7',
           'flux_net_forcing_5','flux_net_forcing_6','flux_net_forcing_7',
           'flux_net_forcing_9','flux_net_forcing_10','flux_net_forcing_11',
           'flux_net_forcing_12','flux_net_forcing_13','flux_net_forcing_14',
           'flux_net_forcing_15','flux_net_forcing_16','flux_net_forcing_17',
           'flux_net_forcing_18']
# level indices for each component 
# (e.g., 0 for surface, 41 for Garand TOA)
# one dictionary key per component so each component
# can have its own set of level indices
CFLEVS = {}
CFLEVS['flux_net'] = [0, 26, 42]
CFLEVS['band_flux_net'] = [42]
CFLEVS['heating_rate'] = range(42)
CFLEVS['heating_rate_7'] = range(42)
CFLEVS['flux_net_forcing_5'] = [0, 26, 42]
CFLEVS['flux_net_forcing_6'] = [0, 26, 42]
CFLEVS['flux_net_forcing_7'] = [0, 26, 42]
CFLEVS['flux_net_forcing_9'] = [0, 26, 42]
CFLEVS['flux_net_forcing_10'] = [0, 26, 42]
CFLEVS['flux_net_forcing_11'] = [0, 26, 42]
CFLEVS['flux_net_forcing_12'] = [0, 26, 42]
CFLEVS['flux_net_forcing_13'] = [0, 26, 42]
CFLEVS['flux_net_forcing_14'] = [0, 26, 42]
CFLEVS['flux_net_forcing_15'] = [0, 26, 42]
CFLEVS['flux_net_forcing_16'] = [0, 26, 42]
CFLEVS['flux_net_forcing_17'] = [0, 26, 42]
CFLEVS['flux_net_forcing_18'] = [0, 26, 42]

# weights for each cost function component
CFWGT = [0.6, 0.04, 0.12, 0.12,
         0.01, 0.02, 0.04,
        0.005, 0.005, 0.005,
        0.005, 0.005, 0.005, 
        0.005, 0.005, 0.005,
        0.005]

# directory under which to store k-distribution files that optimize 
# the cost function for each iteration and diagnistics (if necessary)
CFDIR = 'fullCF_top-layer_redo_abs_parabola'
FCC.pathCheck(CFDIR, mkdir=True)

Now we starting combining _g_-points and calculating costs for all combinations, then select the trial that minimizes the cost for a given iteration. Rinse and repeat.

In [ ]:
### write diagnostic netCDFs with cost function components
DIAGNOSTICS = True

RESTORE = True

if RESTORE:
    assert os.path.exists(pickleCost), 'Cannot find {}'.format(pickleCost)
    print('Restoring {}'.format(pickleCost))
    with open(pickleCost, 'rb') as fp: coObj = pickle.load(fp)
else:
    # instantiate object for computing cost
    coObj = REDUX.gCombine_Cost(
        kBandDict, fullBandFluxes, REFNC, TESTNC, 1, 
        DOLW, profilesNC=GARAND, exeRRTMGP=EXE, 
        costFuncComp=CFCOMPS, costFuncLevs=CFLEVS, 
        costWeights=CFWGT, optDir='./{}'.format(CFDIR))
# endif RESTORE

# number of iterations for the optimization
NITER = 94

COSTCUT = 0.1

for i in range(coObj.iCombine, NITER+1):
    print('Iteration {}'.format(i))
    coObj.kMap()
    coObj.fluxComputePool()
    coObj.fluxCombine()
    if i == 1: coObj.costFuncComp(init=True)
    coObj.costFuncComp()
    coObj.findOptimal()
    if coObj.optimized: break
    if DIAGNOSTICS: coObj.costDiagnostics()

    # Start of special g-point combination branch
    dCostIter = np.abs(coObj.totalCost[coObj.iOpt]-coObj.winnerCost)
    if dCostIter > COSTCUT: break

    # coObj = REDUX.modCombine(coObj, i, coObj.optBand, 
    #     diagnostics=DIAGNOSTICS, 
    #     kDirIn=BANDSPLITDIR, fluxDirIn=FULLBANDFLUXDIR)

    coObj.setupNextIter()
    with open(pickleCost, 'wb') as fp: pickle.dump(coObj, fp)
    if i in [90, 93, 100, 114]:
        with open('LW_cost-optimize_iter{:03d}.pickle'.format(i), 'wb') as fp: pickle.dump(coObj, fp)
    coObj.calcOptFlux(
        fluxOutNC='optimized_{}_fluxes_iter{:03d}.nc'.format(DOMAIN, i))
# end iteration loop

KOUTNC = 'rrtmgp-data-{}-g-red.nc'.format(DOMAIN)

# TO DO: getting this error:
# `Using a DataArray object to construct a variable is ambiguous, please extract the data using the .data property.`
# but not sure how to fix -- downgrading xarray didn't work
# it's not vital, though, for now -- we have all the information to RESTORE then try building this 
# again when we think we have a fix, as long as the files are not deleted
# coObj.kDistOpt(KFULLNC, kOutNC=KOUTNC)

coObj.calcOptFlux(fluxOutNC='optimized_{}_fluxes.nc'.format(DOMAIN))
with open('LW_cost-optimize_iter{:03d}.pickle'.format(i), 'wb') as fp: pickle.dump(coObj, fp)

# Modified _g_-Point Combining

At this point, we have reached large enough delta-costs -- i.e., the cost difference with respect to the **previous** iteration -- such that we would like to implement the modified _g_-point combining [described](https://github.com/pernak18/g-point-reduction/wiki/Modified-g-Point-Combining) by Eli. [Issue 19](https://github.com/pernak18/g-point-reduction/issues/19) details the refinements on this modified combination approach, namely:

- parabolas are not necessary used in finding the zero crossing anymore; Eli found that the trends of scaling the weights and their associated delta-costs were approximately linear
- we need to focus on all remaining trials, not just the "winner" of a given iteration as we did in Karen's implementation of the modified combining (methods with `sgl` in their names)

The latter item is the stickler because we have to calculate fluxes and associated costs for all remaining trials over a few more "weight scales" that are the abscissa to go along with their delta-cost ordinates, which will be used in determining zero crossings and thus the scale weight that minimizes the delta-cost. Currently, we scale the _g_-point weights with `iniWgt` scaled with 0 ("init"), 1 ("plus"), and 2 ("2plus"), which is the naming convention used by Karen. We may revisit how many data points we use, given that a) the flux and cost computations are expensive (time), and b) delta-costs are linear with weight. 

First, we define some variables that are exclusive to modified combining:

In [ ]:
with open('LW_cost-optimize_iter094.pickle', 'rb') as fp: coObj = pickle.load(fp)

iniWgt = 0.05
kModPickle = 'LW_mod-k-dist.pickle'
costModPickle = 'LW_mod-cost-optimize.pickle'
kBand = coObj.distBands

Then we have to apply the modified combinations and generate their corresponding _k_-distribution file:

In [ ]:
kBandDict = MODRED.kModInit(coObj.distBands, coObj.iCombine, doLW=DOLW, 
    weight=iniWgt, kDir=BANDSPLITDIR, fluxDir=FULLBANDFLUXDIR)

with open(kModPickle, 'wb') as fp: pickle.dump(kBandDict, fp)
print('Wrote modified k-dist objects to {}'.format(kModPickle))

Now, we continue with those _k_-distributions in flux and cost computation, which is done exactly like we did with the original combinations. Note for diagnostics, we need to sync the modified cost optimization object (`coObjMod`) with original cost-optimization object (`coObj`) up to the current iteration where modification is first applied. Then we save the delta-costs for all trials in all weight scales to be used in the zero-crossing calculation.

In [ ]:
dCostMod, coObjMod = MODRED.costModInit(coObj, kBandDict)
with open(costModPickle, 'wb') as fp: pickle.dump(coObjMod, fp)

In [ ]:
newScales, cross = MODRED.scaleWeightRegress(dCostMod)

Another round of _k_-distribution modifications to create the netCDFs with the zero-crossing scale weights that were found in the linear regression:

In [ ]:
coObjMod = MODRED.whereRecompute(kBand, coObjMod, cross, newScales, weight=iniWgt)
coObjRep = MODRED.recompute(coObj, coObjMod, np.where(cross)[0])

Now we need to consolidate the reprocessed costs with modified-but-not-reprocessed (i.e., no zero crossing) trials. Then we can find the optimal solution after the full suite of _g_-point modification has been applied.

In [ ]:
coObjMod = MODRED.trialConsolidate(coObjMod, coObjRep, np.where(cross)[0], coObj.winnerCost)

# save modified object WITH reprocessed results
costRepPickle = costModPickle.replace('mod', 'rep')
with open(costRepPickle, 'wb') as fp: pickle.dump(coObjMod, fp)

Now that one iteration with modified combining is complete, we can proceed similarly to more iterations. This has the potentially to be **very** time consuming, even parallelized on a supercomputer (because of I/O? Python parallelization overhead? unsure why...), so from here on out, we'll try to use the _k_-distribution and cost optimization objects we've already taken the time to generate and only replace trials for the band that contained the winner in the previous iteration. While we're doing this only with modified combining, there is no reason this logic cannot be applied to iterations where the modified combining is not applied.

While code snippets that follow work, they are poorly organized. In the future, one task may be to streamline the snippets a little more.

A note on the number of _g_-points that should exist at this point: e.g., at iteration 94, after combination modification, band 12 has a winner in it. Band 12 has not had a winner since iteration 78. At that iteration, 8 _g_-points were left in the band. That number perpetuates at iteration 94 because we have not done any additional combining of points -- we've only _changed_ how the 8 points were combined (with weights and scales to the weights). I'm not sure how this will affect future iterations where a new set of combinations is generated...I believe we'll have _x_ trials that need to be recomputed, but only _x_-1 combinations, so i'll just have to use `pop` or `totalCost`, `dCost`, etc. filters for NaNs to calibrate the trial numbers for the current iteration.

In [ ]:
# TO DO: verify *setupNextIter() (did we preserve what we wanted and add what we wanted?)
# possible verification code snippet?
# for i, fia in enumerate(coObj.fluxInputsAll):
#     print(i, fia['bandID'], coObjMod.optBand, coObj.totalCost[i], coObj.dCost[i], coObjMod.totalCost[i], coObjMod.dCost[i], len(coObjMod.fluxInputsAll[i].keys()))

# also compare initial modified trial costs to subsequent winners for verification
metric = coObjMod.totalCost-coObj.winnerCost
iSort = np.argsort(np.abs(metric))
dcmi = dCostMod['init']
dcm1 = dCostMod['plus']
dcm2 = dCostMod['2plus']
import pandas as PD
temp = []
for iisort, isort in enumerate(iSort): temp.append([iisort, isort, coObjMod.fluxInputsAll[isort]['bandID']+1, newScales[isort], dcmi[isort], dcm1[isort], dcm2[isort], metric[isort]])

In [ ]:
PD.DataFrame(temp, columns=('index', 'trial', 'band', 'weight scale', 'dCost 0', 'dCost 1', 'dCost 2', 'dCost')).to_csv('dCost_mods_iter094_corrected2.csv')

In [ ]:
RESTORE = False

if RESTORE:
    costModPickle = 'LW_mod-cost-optimize.pickle'
    with open(costModPickle, 'rb') as fp: coObjMod = pickle.load(fp)
    iniWgt = 0.05
    kBandDict = MODRED.kModInit(coObjMod.distBands, coObjMod.iCombine, 
        doLW=DOLW, weight=iniWgt, kDir=BANDSPLITDIR, fluxDir=FULLBANDFLUXDIR)
else:
    MODRED.trialConsolidate(coObjMod, coObjRep, np.where(cross)[0], coObj.winnerCost)
    coObjMod.iCombine = coObj.iCombine
# endif RESTORE

In [ ]:
# with open('LW_cost-optimize_iter094.pickle', 'rb') as fp: coObj = pickle.load(fp)

# need to recalculate the fluxes and costs with new winner
# recalibrated object will contain all trials
# should only need to be used in subsequent winner choices
coObjRC = MODRED.recalibrate(coObjMod)

In [ ]:
from copy import deepcopy as DCP

# kFiles will likely just contain plus and 2plus (no init), because we're always doing 
# the modified combination at this point
kFiles = {}
for sWgt in ['plus', '2plus']:
    objModCP = DCP(coObjMod)
    bandStr = 'band{:02d}'.format(coObjMod.optBand+1)
    kBandDict[sWgt][bandStr], kFiles[sWgt] = MODRED.kModSetupNextIter(
      objModCP, iniWgt, scaleWeight=sWgt)
    coObjMod.distBands[bandStr] = kBandDict[sWgt][bandStr]
# end scaleWeight loop

In [ ]:
prevWinCost = coObjRC.winnerCost
# coObj0 = DCP(coObjMOd)
coObjNew, bandCosts, idModTrial = MODRED.coModSetupNextIter(coObjMod, list(idModTrial))

In [ ]:
for i, dc in enumerate(dCostNew['init']): print(i+1, dCostMod['init'][i], dc)
# dCostMod, coObjMod = MODRED.costModInit(coObj, kBandDict)


In [ ]:
# TO DO: allow restart with modified reduction

# save necessary variables we haven't already pickled so we don't have to run previous steps every time
# this iteration
modParamsPickle = 'modified_reduction_parameters_iter{:03d}.pickle'.format(coObjNew.iCombine-1)
modParams = {
    'dcost_dict': dCostMod, 'scale_factors': newScales, 
    'zero_cross': cross, 'k_files': kFiles, 'k_objects': kBandDict, 
    'band_costs': bandCosts}
with open(modParamsPickle, 'wb') as fp: pickle.dump(modParams, fp)

with open('LW_mod-cost-optimize_for_iter{:03d}.pickle'.format(coObjNew.iCombine), 'wb') as fp: pickle.dump(coObjNew, fp)
with open('LW_mod-cost-optimize_recal_iter{:03d}.pickle'.format(coObjNew.iCombine-1), 'wb') as fp: pickle.dump(coObjRC, fp)

In [ ]:
#### restore the variables
RESTOREMODPARAMS = True
if RESTOREMODPARAMS:
    modParamsPickle = 'modified_reduction_parameters_iter094.pickle'
    with open(modParamsPickle, 'rb') as fp:
        modParams = pickle.load(fp)
        dCostMod = modParams['dcost_dict']
        newScales = modParams['scale_factors']
        cross = modParams['zero_cross']
        kFiles = modParams['k_files']
        kBandDict = modParams['k_objects']
        bandCosts = modParams['band_costs']
    # endwith
    with open('LW_mod-cost-optimize_for_iter095.pickle', 'rb') as fp: coObjNew = pickle.load(fp)
    with open('LW_mod-cost-optimize_recal_iter094.pickle', 'rb') as fp: coObjRC = pickle.load(fp)
    with open('LW_cost-optimize_iter094.pickle', 'rb') as fp: coObj = pickle.load(fp)
    prevWinCost = coObjRC.winnerCost
    iniWgt = 0.05
# endif RESTORE MODPARAMS

In [ ]:
from copy import deepcopy as DCP

RESTOREMODPARAMS = True
if RESTOREMODPARAMS:
    modParamsPickle = 'modified_reduction_parameters_iter095.pickle'
    with open(modParamsPickle, 'rb') as fp:
        # modParams = {'k_files': kFiles, 'k_objects': kBandDict, 'band_costs': bandCosts, 'dcost_dict_next': dCostNew}
        modParams = pickle.load(fp)
        kFiles = modParams['k_files']
        kBandDict = modParams['k_objects']
        bandCosts = modParams['band_costs']
        dCostMod = modParams['dCost_mod']
    # endwith
    with open('LW_mod-cost-optimize_for_iter096.pickle', 'rb') as fp: coObjNew = pickle.load(fp)

    prevWinCost = coObjNew.winnerCost
    iniWgt = 0.05

    with open('LW_cost-optimize_iter094.pickle', 'rb') as fp: coObj = pickle.load(fp)
# endif RESTORE MODPARAMS

for i in tqdm(range(coObjNew.iCombine, coObjNew.iCombine+2)):
    print('Iteration {}'.format(i))

    iBandTrials, bandObj, dCostMod = MODRED.doBandTrials(coObjNew, kFiles, bandCosts, dict(dCostMod))
    MODRED.trialConsolidate(coObjNew, bandObj, iBandTrials, prevWinCost)
    # TO DO: coObj.winnerCost is the wrong offset after a band contains a winner
    coObjRC, dCostMod = MODRED.rootUpdate(coObjNew, dCostMod, coObj.winnerCost, iBandTrials)

    with open('LW_cost-optimize_iter{:03d}.pickle'.format(i), 'wb') as fp: pickle.dump(coObjRC, fp)

    kFiles = {}
    for sWgt in ['plus', '2plus']:
        objModCP = DCP(coObjRC)
        bandStr = 'band{:02d}'.format(coObjRC.optBand+1)
        print(sWgt, bandStr)
        kBandDict[sWgt][bandStr], kFiles[sWgt] = MODRED.kModSetupNextIter(
          objModCP, iniWgt, scaleWeight=sWgt)
        coObjRC.distBands[bandStr] = kBandDict[sWgt][bandStr]
    # end scaleWeight loop

    coObj0 = DCP(coObjRC)
    coObjNew, bandCosts, dCostMod = MODRED.coModSetupNextIter(coObj0, dCostMod)
    prevWinCost = coObjNew.winnerCost

    with open('LW_mod-cost-optimize_for_iter{:03d}.pickle'.format(coObjNew.iCombine), 'wb') as fp: pickle.dump(coObjNew, fp)

    modParamsPickle = 'modified_reduction_parameters_iter{:03d}.pickle'.format(coObjNew.iCombine-1)
    modParams = {'k_files': kFiles, 'k_objects': kBandDict, 'band_costs': bandCosts, 'dCost_mod': dCostMod}
    with open(modParamsPickle, 'wb') as fp: pickle.dump(modParams, fp)    
# end combination loop

In [ ]:
coeffs = np.polyfit([0, 1, 2], [-1.41312198e-03,  1.72157582e+00,  3.55118931e-01], 2)
coeffs

In [ ]:
roots = np.roots(coeffs)
roots

In [280]:
# this process would be more accurate, but it's VERY slow (~10-12 minutes 
# per iteration), blows up the disk space, and produces a 
# "LinAlgError: SVD did not converge in Linear Least Squares" on iter 95

with open('LW_cost-optimize_iter150.pickle', 'rb') as fp: coObjNew = pickle.load(fp)
iniWgt = 0.05

# FIX THIS ERROR AFTER 150
# wanna get up to 176, which would be iCombine + 82, but let's do 10-ish 
# chunks at a time so we can post results (standard output)
for i in tqdm(range(coObjNew.iCombine, coObjNew.iCombine+26)):
    print()
    coObjNew = MODRED.repeat_mod_redux(coObjNew, doLW=DOLW, iniWgt=iniWgt, 
      fullBandFluxDir=FULLBANDFLUXDIR, fullBandkDir=BANDSPLITDIR)
    # if i % 5 == 0:
    #   with open('LW_cost-optimize_iter{:03d}.pickle'.format(i), 'wb') as fp: pickle.dump(coObjNew, fp)
    with open('LW_cost-optimize_iter{:03d}.pickle'.format(i), 'wb') as fp: pickle.dump(coObjNew, fp)


  0%|          | 0/26 [00:00<?, ?it/s]


band13_coefficients_LW_g03-04_iter151_2p151.nc, Trial: 86, Cost: 105.125170, Delta-Cost: 0.7632
	flux_net, band_flux_net, heating_rate, heating_rate_7, flux_net_forcing_5, flux_net_forcing_6, flux_net_forcing_7, flux_net_forcing_9, flux_net_forcing_10, flux_net_forcing_11, flux_net_forcing_12, flux_net_forcing_13, flux_net_forcing_14, flux_net_forcing_15, flux_net_forcing_16, flux_net_forcing_17, flux_net_forcing_18 = 106.8802, 104.0238, 97.5369, 103.7645, 102.9103, 98.3291, 119.1113, 97.8040, 82.7138, 100.8565, 101.0959, 101.2410, 99.9596, 98.3183, 98.2247, 101.6861, 102.0614


  4%|▍         | 1/26 [07:58<3:19:17, 478.31s/it]


band01_coefficients_LW_g02-03_iter152_2p152.nc, Trial: 2, Cost: 104.266787, Delta-Cost: -0.8584
	flux_net, band_flux_net, heating_rate, heating_rate_7, flux_net_forcing_5, flux_net_forcing_6, flux_net_forcing_7, flux_net_forcing_9, flux_net_forcing_10, flux_net_forcing_11, flux_net_forcing_12, flux_net_forcing_13, flux_net_forcing_14, flux_net_forcing_15, flux_net_forcing_16, flux_net_forcing_17, flux_net_forcing_18 = 105.1257, 102.0804, 98.5763, 104.9921, 102.9103, 98.3291, 119.1118, 97.8040, 82.7138, 100.8565, 101.0959, 101.2410, 99.9596, 98.3183, 98.2247, 101.6861, 102.0614


  8%|▊         | 2/26 [16:50<3:23:54, 509.75s/it]

...

 92%|█████████▏| 24/26 [3:15:38<15:42, 471.23s/it]


band07_coefficients_LW_g01-02_iter175_regr175.nc, Trial: 40, Cost: 104.998279, Delta-Cost: -1.1978
	flux_net, band_flux_net, heating_rate, heating_rate_7, flux_net_forcing_5, flux_net_forcing_6, flux_net_forcing_7, flux_net_forcing_9, flux_net_forcing_10, flux_net_forcing_11, flux_net_forcing_12, flux_net_forcing_13, flux_net_forcing_14, flux_net_forcing_15, flux_net_forcing_16, flux_net_forcing_17, flux_net_forcing_18 = 92.1135, 140.7142, 113.3751, 118.4835, 107.9142, 96.6367, 208.0824, 113.9464, 75.8906, 99.3546, 72.5077, 100.6228, 100.2148, 106.9003, 116.5393, 101.3898, 101.3109


 96%|█████████▌| 25/26 [3:23:26<07:50, 470.08s/it]


band10_coefficients_LW_g03-04_iter176_2p176.nc, Trial: 53, Cost: 105.012555, Delta-Cost: 0.0143
	flux_net, band_flux_net, heating_rate, heating_rate_7, flux_net_forcing_5, flux_net_forcing_6, flux_net_forcing_7, flux_net_forcing_9, flux_net_forcing_10, flux_net_forcing_11, flux_net_forcing_12, flux_net_forcing_13, flux_net_forcing_14, flux_net_forcing_15, flux_net_forcing_16, flux_net_forcing_17, flux_net_forcing_18 = 91.8797, 147.6579, 113.2575, 117.5749, 107.9142, 96.6367, 208.0824, 113.9464, 75.8906, 99.3546, 72.5077, 100.6228, 100.2148, 106.9003, 116.5393, 101.3898, 101.3109


100%|██████████| 26/26 [3:32:10<00:00, 489.62s/it]


In [265]:
  # with open('LW_cost-optimize_iter151.pickle', 'rb') as fp: inObj = pickle.load(fp)

  from copy import deepcopy as DCP
  inObj = DCP(coObjNew)

  kBand = inObj.distBands

  # basically, this is every step that i did after iteration 94
  # and before iteration 95
  kBandDict = MODRED.kModInit(inObj.distBands, inObj.iCombine, 
    doLW=DOLW,  weight=iniWgt, 
    kDir=BANDSPLITDIR, fluxDir=FULLBANDFLUXDIR)

  # print(len(inObj.totalCost))
  dCostMod, coObjMod = MODRED.costModInit(inObj, kBandDict)
  newScales, cross = MODRED.scaleWeightRegress(dCostMod)

In [266]:
  coObjMod = MODRED.whereRecompute(
    kBand, coObjMod, cross, newScales, weight=iniWgt)
  # print(len(coObjMod.totalCost))
  coObjRep = MODRED.recompute(inObj, coObjMod, np.where(cross)[0])
  coObjMod = MODRED.trialConsolidate(
    coObjMod, coObjRep, np.where(cross)[0], inObj.winnerCost)
  # print(len(coObjMod.totalCost))

  # print(len(coObjMod.totalCost))
  coObjMod.iCombine = inObj.iCombine
  coObjRC = MODRED.recalibrate(coObjMod)
  # print(len(coObjMod.totalCost))

  # kFiles will likely just contain plus and 2plus (no init), 
  # because we're always doing the modified combination at this point
  kFiles = {}
  for sWgt in ['plus', '2plus']:
    objModCP = DCP(coObjMod)
    bandStr = 'band{:02d}'.format(coObjMod.optBand+1)
    kBandDict[sWgt][bandStr], kFiles[sWgt] = MODRED.kModSetupNextIter(
      objModCP, iniWgt, scaleWeight=sWgt)
    coObjMod.distBands[bandStr] = kBandDict[sWgt][bandStr]
  # end scaleWeight loop

  temp, bandCosts, tempDCM = MODRED.coModSetupNextIter(coObjMod, dict(dCostMod))

85 None 0 1 ./fullCF_top-layer_redo_abs_parabola/band15_coefficients_LW_g01-02_iter145_2p145.nc
band07_coefficients_LW_g01-02_iter153_2p153.nc, Trial: 48, Cost: 103.519969, Delta-Cost: -0.7468
	flux_net, band_flux_net, heating_rate, heating_rate_7, flux_net_forcing_5, flux_net_forcing_6, flux_net_forcing_7, flux_net_forcing_9, flux_net_forcing_10, flux_net_forcing_11, flux_net_forcing_12, flux_net_forcing_13, flux_net_forcing_14, flux_net_forcing_15, flux_net_forcing_16, flux_net_forcing_17, flux_net_forcing_18 = 103.2441, 105.6427, 99.7353, 105.5078, 102.9103, 98.3291, 120.0781, 97.8040, 82.7138, 100.8565, 101.0959, 101.2410, 99.9596, 98.3183, 98.2247, 101.6861, 102.0614


In [275]:
  temp, bandCosts, tempDCM = MODRED.coModSetupNextIter(coObjMod, dict(dCostMod))
bandCosts

array([103.20283264, 102.89373098, 102.57125175, 106.63945285,
       115.9818273 , 107.30128503])

In [279]:
  iBandTrials, bandObj, _ = MODRED.doBandTrials(
    temp, kFiles, bandCosts, dict(dCostMod))
  MODRED.trialConsolidate(temp, bandObj, iBandTrials, temp.winnerCost)

0 6 0 1 ./fullCF_top-layer_redo_abs_parabola/band07_coefficients_LW_g01-02_iter153_2p153.nc


In [ ]:
with open('LW_cost-optimize_iter151.pickle', 'rb') as fp: inObj = pickle.load(fp)
inObj.totalCost

In [220]:
with open('LW_cost-optimize_iter{:03d}.pickle'.format(i), 'wb') as fp: pickle.dump(temp, fp)

In [8]:
with open('LW_cost-optimize_iter176.pickle', 'rb') as fp: coObjNew = pickle.load(fp)
KOUTNC = 'rrtmgp-data-{}-g-red.nc'.format(DOMAIN)
coObjNew.kDistOpt(KFULLNC, kOutNC=KOUTNC)

coObjNew.calcOptFlux(fluxOutNC='optimized_{}_fluxes.nc'.format(DOMAIN))
with open('LW_cost-optimize_iter{:03d}.pickle'.format(i), 'wb') as fp: pickle.dump(coObj, fp)

FileNotFoundError: [Errno 2] No such file or directory: b'/global/project/projectdirs/e3sm/pernak18/reference_netCDF/g-point-reduce/rrtmgp-data-lw-g256-jen-xs.nc'